In [20]:
######################
# Process MIDI Files #
######################

from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob
import json
import os

# VARS
DATA_GLOB = "data/pokemon/PkmRB/*.mid"
MODEL_DIR = "model/pokemon/PkmRB-V1/"
NOTES_FILE = "%s/%s" % (MODEL_DIR, "notes.json")

NOTES_BY_FILE = []
NOTES = set()

for input_file in glob.glob(DATA_GLOB):
    print("Ingesting... %s" % input_file)
    
    midi = converter.parse(input_file)
    parts = instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    
    file_notes = []
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            file_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            file_notes.append('.'.join(str(n) for n in element.normalOrder))
    NOTES_BY_FILE.append(file_notes)
    NOTES.update(set(file_notes))

NOTES = sorted(NOTES)

if not os.path.exists(os.path.dirname(NOTES_FILE)):
    os.makedirs(os.path.dirname(NOTES_FILE))
with open(NOTES_FILE, 'w') as note_file:
    json.dump(NOTES, note_file)

Ingesting... data/pokemon/PkmRB/PkmRB-Battle3.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Evolving.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Celadon.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Victory1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-10.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Pallet.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Caught.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Cinnabar.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Battle1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Victory2.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Enc1.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Rival.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Lavender.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Mansion.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Hideout.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Pokedex.mid
Ingesting... data/pokemon/PkmRB/PkmRB-24.mid
Ingesting... data/pokemon/PkmRB/PkmRB-Vermilion.mid
Ingesting... data/pokemon/PkmRB/PkmRB-12.mid
Ingesting... data/pokemon/PkmRB/PkmRB-1.mid
Ingesting... data/pok

In [21]:
########################
# BUILD NOTE SEQUENCES #
########################

import numpy
from keras.utils import np_utils

NOTES_FILE = None
SEQUENCE_LENGTH = 100
X = []
y = []

# Load Notes from disk if desired
if NOTES_FILE:
    with open(NOTES_FILE, 'r') as notes_file:
        NOTES = json.loads(notes_file.read())

NOTE_TO_INT = dict((note, number) for number, note in enumerate(NOTES))
INT_TO_NOTE = dict((number, note) for number, note in enumerate(NOTES))

for notes in NOTES_BY_FILE:
    for i in range(0, len(notes) - SEQUENCE_LENGTH, 1):
        input_sequence = notes[i:i + SEQUENCE_LENGTH] # N Input Notes
        output_sequence = notes[i + SEQUENCE_LENGTH] # 1 Output Note
        
        X.append([NOTE_TO_INT[note] for note in input_sequence])
        y.append(NOTE_TO_INT[output_sequence])
        
N_SEQUENCES = len(X)
N_VOCAB = len(NOTES)

print("Num Sequences: %d" % N_SEQUENCES)
print("Num Vocab: %d" % N_VOCAB)

X = numpy.reshape(X, (N_SEQUENCES, SEQUENCE_LENGTH, 1)) / N_VOCAB
y = np_utils.to_categorical(y)

Num Sequences: 24664
Num Vocab: 56


In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

MODEL_FILE = "%s/%s" % (MODEL_DIR, "weights-improvement-82-0.2081.hdf5")
CHECKPOINT_FILE = MODEL_FILE

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(N_VOCAB))
model.add(Activation('softmax'))
if (CHECKPOINT_FILE):
    print("Loading Checkpoint File: %s" % CHECKPOINT_FILE)
    model.load_weights(CHECKPOINT_FILE)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Loading Checkpoint File: model/pokemon/PkmRB-V1//weights-improvement-82-0.2081.hdf5


In [32]:
import uuid
from music21 import note, chord, stream

MIDI_OUTPUT_DIRECTORY = "output/pokemon/PkmRB-V1/"

SEED_INPUT_SEQUENCE = X[numpy.random.randint(0, len(X))] * N_VOCAB
SEED_SEQUENCE = [INT_TO_NOTE[numpy.round(number[0])] for number in SEED_INPUT_SEQUENCE]

print('|'.join(SEED_SEQUENCE))

pattern = SEED_INPUT_SEQUENCE
output = []

for i in range(500):
    input_sequence = numpy.reshape(pattern, (1, len(pattern), 1))
    output_sequence = model.predict(input_sequence, verbose=0)
    
    number = numpy.argmax(output_sequence)
    result = INT_TO_NOTE[number]
    output.append(result)
    
    pattern = numpy.append(pattern, number)
    pattern = pattern[1:]
    
offset = 0
output_notes = []
for note_pattern in output:
    if ('.' in note_pattern) or note_pattern.isdigit():
        notes_in_chord = note_pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note = storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(note_pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

if not os.path.exists(os.path.dirname(MIDI_OUTPUT_DIRECTORY)):
    os.makedirs(os.path.dirname(MIDI_OUTPUT_DIRECTORY))
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp="%s/%s.%s.%s" % (MIDI_OUTPUT_DIRECTORY, os.path.basename(CHECKPOINT_FILE), uuid.uuid4(), "mid"))

E4|B4|E4|C#4|G#4|B4|E4|B4|G#4|E5|E4|B4|E4|B4|G#4|E4|B4|E4|E4|B4|G#4|B4|E4|B4|E4|B4|C#4|G4|C#4|A4|C#4|A3|E4|A4|C#4|G4|E4|C#5|C#4|A4|C#4|A4|E4|C#4|A4|C#4|A4|C#4|E4|A4|C#4|A4|G4|C#5|C#4|A4|F#4|D5|D4|A4|D4|E4|A4|F#4|A4|D4|A4|D4|E5|A4|G4|D5|D4|A4|D4|A4|E4|G4|A4|F#4|G4|A4|B4|E4|C#5|A4|D5|F#4|A4|C#5|F#4|F#4|B4|F#4|G4|C#5|G4|G4|B4|G4


'output/pokemon/PkmRB-V1//weights-improvement-82-0.2081.hdf5.66221685-8f4e-45d1-b35f-0205053ca626.mid'